# Algoritmo de Colonia de Hormigas para Coloreado de Grafos

Pablo Quiroz Apud

German Garcia Beltran


In [6]:
import numpy as np
import random
from collections import defaultdict

# Parámetros
max_cycles = 100
max_ants = 100
alpha = 1
beta = 4
ro = 0.5

file = "edges_100_04.txt"

graph_neighbors = defaultdict(set) 
non_neighbors = defaultdict(set)
vertices = set()
pheromone_matrix = {}
pheromone_updates = {}
current_coloring = defaultdict(set)
best_coloring = defaultdict(set)

def load_graph(filename):
    global vertices, graph_neighbors
    with open(filename, 'r') as file:
        for line in file:
            node1, node2 = map(int, line.split())
            graph_neighbors[node1].add(node2)
            graph_neighbors[node2].add(node1)
            vertices.update({node1, node2})

def initialize_non_neighbors():
    global non_neighbors
    for vertex in vertices:
        non_neighbors[vertex] = vertices - {vertex} - graph_neighbors[vertex]

def initialize_pheromone_matrix():
    global pheromone_matrix
    for vertex1 in vertices:
        for vertex2 in non_neighbors[vertex1]:
            pheromone_matrix[(vertex1, vertex2)] = 1

def reset_pheromone_updates():
    global pheromone_updates
    pheromone_updates = {(v1, v2): 0 for v1 in vertices for v2 in non_neighbors[v1]}

def count_used_colors(coloring):
    return sum(1 for color_set in coloring.values() if len(color_set) > 0)

def update_dsat_scores(uncolored_set, vertex, coloring, max_colors, dsat_scores):
    for neighbor in uncolored_set.intersection(graph_neighbors[vertex]):
        dsat_scores[neighbor] = len(
            {color for color in range(1, max_colors + 1) if graph_neighbors[neighbor].intersection(coloring[color])}
        )

def update_min_colors(vertex, uncolored_set, max_colors, min_colors):
    for neighbor in graph_neighbors[vertex].intersection(uncolored_set):
        for color in range(min_colors[neighbor], max_colors + 2):
            if not graph_neighbors[neighbor].intersection(current_coloring[color]):
                min_colors[neighbor] = color
                break

def select_next_vertex(uncolored_set, dsat_scores, min_colors):
    probabilities = []
    total_weight = 0
    for vertex in uncolored_set:
        pheromone_sum = sum(
            pheromone_matrix.get((neighbor, vertex), 0)
            for neighbor in current_coloring[min_colors[vertex]]
        ) / max(1, len(current_coloring[min_colors[vertex]]))
        weight = (dsat_scores[vertex] ** beta) * (pheromone_sum ** alpha)
        probabilities.append(weight)
        total_weight += weight

    if total_weight == 0:
        return random.choice(list(uncolored_set))

    probabilities = [weight / total_weight for weight in probabilities]
    return random.choices(list(uncolored_set), probabilities)[0]

def update_pheromone_updates(best_coloring, num_colors):
    for color in range(1, num_colors + 1):
        for node1 in best_coloring[color]:
            for node2 in best_coloring[color]:
                if node1 != node2 and node1 not in graph_neighbors[node2]:
                    pheromone_updates[(node1, node2)] += 1 / num_colors

def apply_pheromone_evaporation():
    for (node1, node2), value in pheromone_updates.items():
        pheromone_matrix[(node1, node2)] = (
            ro * pheromone_matrix.get((node1, node2), 0) + value
        )

# Ejecución del programa
load_graph(file)
initialize_non_neighbors()
initialize_pheromone_matrix()

min_colors_used = float('inf')

for cycle_number in range(max_cycles):
    reset_pheromone_updates()
    for ant_number in range(max_ants):
        uncolored_vertices = set(vertices)
        dsat_scores = {vertex: 0 for vertex in vertices}
        min_colors = {vertex: 1 for vertex in vertices}
        current_coloring = defaultdict(set)

        start_vertex = max(uncolored_vertices, key=lambda v: len(graph_neighbors[v]))
        current_coloring[1].add(start_vertex)
        uncolored_vertices.remove(start_vertex)
        num_colors = 1

        while uncolored_vertices:
            update_min_colors(start_vertex, uncolored_vertices, num_colors, min_colors)
            update_dsat_scores(uncolored_vertices, start_vertex, current_coloring, num_colors, dsat_scores)
            start_vertex = select_next_vertex(uncolored_vertices, dsat_scores, min_colors)
            color = min_colors[start_vertex]
            current_coloring[color].add(start_vertex)
            uncolored_vertices.remove(start_vertex)
            if color == num_colors + 1:
                num_colors += 1

        if num_colors < min_colors_used:
            
            best_coloring = current_coloring.copy()
            min_colors_used = num_colors
            print("Ciclo número:", cycle_number)
            print("Hormiga número:", ant_number)
            print("Número mínimo de colores: ", min_colors_used)
            print()

        update_pheromone_updates(best_coloring, num_colors)
    apply_pheromone_evaporation()
    
best_coloring = {k: v for k, v in best_coloring.items() if v}

print("Coloreado Final:", best_coloring)



Ciclo número: 0
Hormiga número: 0
Número mínimo de colores:  19

Ciclo número: 0
Hormiga número: 1
Número mínimo de colores:  18

Ciclo número: 0
Hormiga número: 3
Número mínimo de colores:  17

Ciclo número: 0
Hormiga número: 14
Número mínimo de colores:  16

Ciclo número: 2
Hormiga número: 37
Número mínimo de colores:  15

Ciclo número: 4
Hormiga número: 65
Número mínimo de colores:  14

Coloreado Final: {1: {1, 66, 7, 11, 80, 84, 22, 55}, 2: {32, 33, 4, 37, 43, 51, 87, 23}, 3: {64, 69, 39, 45, 82, 21, 88, 92}, 4: {0, 98, 99, 70, 40, 44, 83, 52, 58}, 5: {35, 5, 77, 46, 48, 19, 61}, 6: {3, 38, 42, 13, 24, 26, 63, 62, 31}, 7: {97, 6, 9, 47, 53, 90, 59, 28}, 8: {72, 74, 79, 50, 20, 85, 89}, 9: {65, 71, 75, 86, 27, 95}, 10: {8, 54, 25, 94, 57}, 11: {34, 67, 12, 18, 60}, 12: {96, 41, 10, 14, 15, 16, 17, 93, 30}, 13: {2, 68, 73, 49, 56}, 14: {36, 76, 78, 81, 91, 29}}
